## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.

1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).

2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  

3. To generate vectors and split the instances into training and testing datasets at random.

4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.

5. To evaluate the performance of the classifier.


You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC



[nltk_data] Downloading package wordnet to
[nltk_data]     /home/gussarayo@GU.GU.SE/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [4]:

inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[0:20]

,Line_No,Sentence_No,Word,POS,NER
0,0,1,thousand,NNS,O
1,1,1,of,IN,O
2,2,1,demonstrator,NNS,O
3,3,1,have,VBP,O
4,4,1,march,VBN,O
5,5,1,through,IN,O
6,6,1,london,NNP,B-geo
7,7,1,to,TO,O
8,8,1,protest,VB,O
9,9,1,the,DT,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [5]:

instances = a2.create_instances(inputdata)

In [6]:
instances[20:40]

[Class: gpe Features: ['<S5>', 'this', 'week', 'restart', 'part', 'the', 'conversion', 'process', 'at', '</S5>'],
 Class: geo Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: gpe Features: ['<S5>', 'official', 'say', 'they', 'expect', 'get', 'access', 'to', 'sealed', 'sensitive'],
 Class: tim Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'surveillance', 'begin', 'function', '</S5>', '</S4>', '</S3>'],
 Class: org Features: ['<S5>', 'surveillance', 'system', 'begin', 'function', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: org Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'backing', 'threaten', 'to', 'refer', '</S5>', '</S4>'],
 Class: gpe Features: ['<S5>', 'backing', 'have', 'threaten', 'to', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: gpe Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'which', 'could', 'impose', 'sanction', 'if'],
 Class: org Features: ['<S3>', '<S4>', '<S5>', 'which', 'could', 'sanction', 'if',

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [7]:

bigdf = a2.create_table(instances)
bigdf[20:40]

,Class,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
20,gpe,5.032928,-1.120715,0.375585,0.116922,0.145641,-0.487438,-0.001267,-0.004255,0.001975,...,-0.000010,0.000022,0.000018,-4.065066e-07,-0.000015,0.000003,0.000006,-0.000004,0.000007,-0.000013
21,gpe,5.989315,0.056117,-0.157471,-0.094129,-0.125747,-0.019278,-0.004521,-0.022386,0.034266,...,0.000350,-0.000840,0.000715,2.269531e-03,-0.001777,0.001041,0.001311,-0.001134,0.000690,0.000399
22,gpe,4.991326,0.047065,-0.132388,-0.079418,-0.106476,-0.016392,-0.003859,-0.019125,0.029345,...,0.000086,-0.000208,0.000184,5.282692e-04,-0.000418,0.000240,0.000315,-0.000274,0.000171,0.000107
23,gpe,5.030752,0.568323,0.758935,0.168941,0.158547,0.022137,-0.021922,0.016067,-0.048622,...,-0.000188,-0.000541,-0.000508,8.019683e-04,-0.000099,0.000333,-0.000024,-0.000592,0.000686,0.000339
24,gpe,7.997703,0.133758,-0.194957,-0.166327,-0.280392,-0.060328,0.018610,-0.101562,0.285300,...,-0.008051,0.006657,0.004575,5.293505e-03,0.002674,0.001781,-0.003306,-0.000855,-0.004270,-0.008155
25,gpe,6.028335,-0.904722,0.224634,0.021182,-0.067715,0.884588,0.006081,0.025655,-0.028096,...,-0.000014,0.000031,0.000028,2.792985e-06,-0.000025,0.000006,0.000011,-0.000008,0.000012,-0.000019
26,gpe,6.000123,0.074781,-0.239641,-0.181075,-0.329846,-0.087698,-0.062631,-0.550162,-0.670491,...,0.000004,-0.000009,0.000008,2.231035e-05,-0.000018,0.000010,0.000014,-0.000012,0.000007,0.000004
27,gpe,5.998953,0.072186,-0.226844,-0.164635,-0.280608,-0.063379,-0.026210,-0.155031,0.548019,...,0.000004,-0.000010,0.000009,2.498480e-05,-0.000020,0.000011,0.000015,-0.000013,0.000008,0.000005
28,gpe,6.012878,0.124895,-0.685803,0.906508,0.261858,0.018889,0.002698,0.011721,-0.011259,...,0.000006,-0.000002,0.000020,7.490371e-05,-0.000017,0.000035,-0.000073,0.000051,-0.000015,-0.000024
29,gpe,6.008186,0.099747,-0.404621,-0.711291,0.869033,0.038004,0.005342,0.023739,-0.027885,...,0.000010,-0.000041,-0.000010,2.755715e-06,-0.000039,-0.000025,0.000059,-0.000026,-0.000091,0.000076


In [8]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 4.64400000e+04,  5.00072496e+00,  6.37810391e-02, ...,
         -9.10247600e-06,  5.72828457e-06,  3.43710639e-06],
        [ 1.54240000e+04,  5.00235838e+00,  1.04188375e-01, ...,
          5.24504545e-03, -8.17139556e-04,  9.42625285e-03],
        [ 6.09850000e+04,  6.02833453e+00, -9.04722127e-01, ...,
         -7.68926193e-06,  1.16402199e-05, -1.86611479e-05],
        ...,
        [ 3.81090000e+04,  5.02801607e+00,  5.12229001e-01, ...,
         -2.87417515e-03,  5.47279472e-03,  1.30527443e-02],
        [ 9.45700000e+03,  6.99883831e+00,  1.23112031e-01, ...,
         -1.64019367e-02,  6.57128783e-03,  2.61758034e-02],
        [ 1.08660000e+04,  6.00012289e+00,  7.47810853e-02, ...,
         -1.16688887e-05,  7.34321183e-06,  4.40657847e-06]]),
 0        org
 1        geo
 2        tim
 3        gpe
 4        tim
         ... 
 55343    per
 55344    geo
 55345    per
 55346    tim
 55347    tim
 Name: Class, Length: 55348, dtype: object,
 array([[ 5.53480000e+04,  4.99

In [9]:
len(test_y) / (len(test_y) + len(train_y))

0.2

In [10]:
len(test_X) / (len(test_X) + len(train_X))

0.2

In [11]:
test_y[0]

'tim'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [12]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

/usr/local/lib64/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [13]:
train_predictions

array(['org', 'gpe', 'org', ..., 'gpe', 'gpe', 'geo'], dtype=object)

In [14]:
train_y

0        org
1        geo
2        tim
3        gpe
4        tim
        ... 
55343    per
55344    geo
55345    per
55346    tim
55347    tim
Name: Class, Length: 55348, dtype: object

In [15]:
test_predictions

array(['org', 'gpe', 'gpe', ..., 'org', 'org', 'eve'], dtype=object)

In [16]:
test_y

0        tim
1        tim
2        geo
3        geo
4        geo
        ... 
13832    tim
13833    tim
13834    tim
13835    tim
13836    tim
Name: Class, Length: 13837, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [17]:
a2.confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,0,0,0,7,0,12,0,1
eve,0,5,0,16,0,26,0,13
geo,0,862,0,1222,0,1811,3,712
gpe,0,423,0,547,0,778,0,232
nat,0,5,0,6,0,7,0,2
org,0,441,0,623,0,887,2,287
per,0,600,0,652,0,1099,4,285
tim,0,467,0,635,0,842,1,322


In [18]:
a2.confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,0,6,272,117,0,37,9,4
eve,0,13,150,116,0,54,4,17
geo,0,1218,4956,6406,0,3008,170,883
gpe,0,632,3564,3602,0,1505,151,329
nat,0,6,81,53,0,19,3,1
org,0,660,3096,3880,0,1693,183,322
per,0,811,2420,3371,0,1824,116,342
tim,0,666,2879,3613,0,1594,90,402


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')